# Typical Pytorch ML Scheme

In [8]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# 1 . Dataset Definition
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    # Assisting information for iterating dataset
    def __len__(self):
        return len(self.data)

    # Work ez with datapipelines like 'DataLoader'
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

# Example data
X = torch.rand(100, 1, 28, 28) # 100 samples of 28x28 images
y = torch.randint(0, 10, (100,)) # Labels for 10 classes

# Dataset and DataLoader

# Use torchvision for data pre processing:
# 1. Data preperation (skaling, convert to tensors)
# 2. Data augmentation (generate more data to prevent overfitting)
# 3. Normalization (tensor-mean)/ std, 0.5 and 0.5 for greyscaled pictures

dataset = CustomDataset(X, y, transform=transforms.Normalize((0.5,), (0.5,)))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 2. Model Definition
class SimpleModel(nn.Module):
    def __init__(self):
        # Call constructor of nn.Module base class
        super(SimpleModel, self).__init__()
        # Define instances of pytorch modules flatten and sequential
        # Register these class attributes as submodules of nn.Module
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 128), # 28 is the image size
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc(x)
        return x

model = SimpleModel()

# 3. Loss and Optimizer

# Multiclass classification:
# Activation fct: Softmax; Loss fct: Cross-Entropy
# Binary Classification:
# Activation fct: Sigmoid; Loss fct: Binary Cross-Entropy
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

# 5. Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")


Epoch [1/5], Loss: 2.3327
Epoch [2/5], Loss: 1.9590
Epoch [3/5], Loss: 1.7173
Epoch [4/5], Loss: 1.5567
Epoch [5/5], Loss: 1.3526
Accuracy: 100.00%
